<a href="https://colab.research.google.com/github/JuniorHZ19/ML/blob/main/alexnet1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#@markdown # *0) ** Instalando libreria(OBLIGATORIO)

!pip install torch torchvision torchaudio

In [17]:
#@markdown # *1) ** Usar el dataset CIFAR 10


import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# Definir transformaciones para normalizar las imágenes
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((227,227)),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# Descargar el conjunto de datos CIFAR-10

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Crear cargadores de datos para facilitar el acceso a lotes de datos

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)




Files already downloaded and verified
Files already downloaded and verified


In [50]:
import torch.nn as nn
import torch.nn.functional as F



class AlexNet(nn.Module):

   def __init__(self):
    super(AlexNet,self).__init__()

    self.conv1=nn.Conv2d(3,96,kernel_size=11,stride=4)
    self.maxpool1=nn.MaxPool2d(kernel_size=3,stride=2)

    self.conv2=nn.Conv2d(96,256,kernel_size=5,stride=1,padding=2)
    self.maxpool2=nn.MaxPool2d(kernel_size=3,stride=2)

    self.conv3=nn.Conv2d(256,384,kernel_size=3,stride=1,padding=2)

    self.conv4=nn.Conv2d(384,384,kernel_size=3,stride=1,padding=2)

    self.conv5=nn.Conv2d(384,256,kernel_size=3,stride=1)
    self.maxpool3=nn.MaxPool2d(kernel_size=3,stride=2)

    self.flatten = nn.Flatten()
    self.lineal1=nn.Linear(7*7*256,4096)
    self.salida=nn.Linear(4096,10)


   def forward(self,x):

     x=self.conv1(x)
     x=F.relu(x)
     x=self.maxpool1(x)

     x=self.conv2(x)
     x=F.relu(x)
     x=self.maxpool2(x)

     x=self.conv3(x)
     x=F.relu(x)

     x=self.conv4(x)
     x=F.relu(x)


     x=self.conv5(x)
     x=F.relu(x)
     x=self.maxpool3(x)



     x=self.flatten(x)


     x=self.lineal1(x)

     out=self.salida(x)

     return out


In [54]:



import torch

if torch.cuda.is_available():
    print("CUDA está disponible.")
    device = torch.device("cuda")
else:
    print("CUDA no está disponible. Se utilizará la CPU.")
    device = torch.device("cpu")

#funcion clasificador:
def clasificador(y_pred_test):
 max_values_tensor=torch.empty((0,),dtype=torch.float32)

 for tensor in y_pred_test:

   max_values,max_indices=torch.max(tensor,dim=0)
   max_values_tensor=torch.cat((max_values_tensor,max_indices.unsqueeze(0)))

 return max_values_tensor

#funcion acutity:
def accurity(y_test,y_real):
   correctos=0
   for predicho, real in zip(y_test, y_real):
    if predicho==real:
     correctos+=1
   return correctos/len(y_test)


import torch.optim as optim



AlexnetRed=AlexNet()

AlexnetRed=AlexnetRed.to(device)


criterio=nn.CrossEntropyLoss()

optimizer=optim.SGD(AlexnetRed.parameters(),lr=0.001)

epochs=100
for epoch in  range(epochs):
 total_train_loss = 0.0
 num_train_batches = len(train_loader)
 AlexnetRed.train()

 for x,y in (train_loader):
  x,y=x.to(device),y.to(device)

  y_predict=AlexnetRed(x)

  loss=criterio(y_predict,y)

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  optimizer.zero_grad()

  total_train_loss+=loss.item()

 AlexnetRed.eval()

 with torch.no_grad():
  total_test_accuracy = 0.0
  num_test_batches = len(test_loader)
  for x_test,y_test in (test_loader):
     x_test,y_test=x_test.to(device),y_test.to(device)
     y_predict_test=AlexnetRed(x_test)
     y_predic_test_clasificado=clasificador(y_predict_test)
     Precision=accurity(y_test,y_predic_test_clasificado)
     total_test_accuracy += Precision
 avg_test_accuracy = total_test_accuracy / num_test_batches
 avg_train_loss=total_train_loss/num_train_batches

print(f'Epoca[{epoch+1}], Perdida: {avg_train_loss:.4f},Accuryty:{avg_test_accuracy:.4f}')



KeyboardInterrupt: ignored

In [ ]:


y_pred_multiclases = torch.tensor([[0.1, 1.8, 0.8, 0.4, 2],
                                   [0.5, 2, 3.5, 5, 2.1]])

def clasificador(y_pred_test):

   max_values_tensor = torch.empty((0,), dtype=torch.float32)

   for tensor in y_pred_test:

     max_values, max_indices  = torch.max(tensor, dim=0)

     max_values_tensor = torch.cat((max_values_tensor, max_indices.unsqueeze(0)))

   return max_values_tensor

print(clasificador(y_pred_multiclases))


tensor([4., 3.])
